In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("joebeachcapital/hotel-reviews")

print("Path to dataset files:", path)

/Users/oceanspiess/PycharmProjects/ML_NLP/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/oceanspiess/PycharmProjects/ML_NLP/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 717M/717M [09:30<00:00, 1.32MB/s]  

Extracting files...


Path to dataset files: /Users/oceanspiess/.cache/kagglehub/datasets/joebeachcapital/hotel-reviews/versions/2


In [7]:
import pandas as pd
import numpy as np

# **1. Préparation du dataset**

In [19]:
path = "/Users/oceanspiess/.cache/kagglehub/datasets/joebeachcapital/hotel-reviews/versions/2/reviews.csv"
df = pd.read_csv(path)

In [14]:
df.info()
df.describe()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878561 entries, 0 to 878560
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ratings            878561 non-null  object
 1   title              878561 non-null  object
 2   text               878561 non-null  object
 3   author             878561 non-null  object
 4   date_stayed        810967 non-null  object
 5   offering_id        878561 non-null  int64 
 6   num_helpful_votes  878561 non-null  int64 
 7   date               878561 non-null  object
 8   id                 878561 non-null  int64 
 9   via_mobile         878561 non-null  bool  
dtypes: bool(1), int64(3), object(6)
memory usage: 61.2+ MB


,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
0,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...","“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,"{'username': 'Papa_Panda', 'num_cities': 22, '...",December 2012,93338,0,2012-12-17,147643103,False
1,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...","{'username': 'Maureen V', 'num_reviews': 2, 'n...",December 2012,93338,0,2012-12-17,147639004,False
2,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Great Stay”,This is a great property in Midtown. We two di...,"{'username': 'vuguru', 'num_cities': 12, 'num_...",December 2012,1762573,0,2012-12-18,147697954,False
3,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,"{'username': 'Hotel-Designer', 'num_cities': 5...",August 2012,1762573,0,2012-12-17,147625723,False
4,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,"{'username': 'JamesE339', 'num_cities': 34, 'n...",December 2012,1762573,0,2012-12-17,147612823,False


In [20]:
# Convert ratings dictionary strings to actual dictionaries
df['ratings'] = df['ratings'].apply(eval)

# Extract individual ratings into separate columns
df['service'] = df['ratings'].apply(lambda x: x.get('service', np.nan))
df['cleanliness'] = df['ratings'].apply(lambda x: x.get('cleanliness', np.nan))
df['overall'] = df['ratings'].apply(lambda x: x.get('overall', np.nan))
df['sleep_quality'] = df['ratings'].apply(lambda x: x.get('sleep_quality', np.nan))
df['location'] = df['ratings'].apply(lambda x: x.get('location', np.nan))
df['rooms'] = df['ratings'].apply(lambda x: x.get('rooms', np.nan))
df['value'] = df['ratings'].apply(lambda x: x.get('value', np.nan))

# Drop the original ratings column if desired
df = df.drop('ratings', axis=1)


In [21]:
# Drop specified columns
columns_to_drop = ['title', 'author', 'date_stayed', 'num_helpful_votes', 'date', 'id', 'via_mobile']
df = df.drop(columns=columns_to_drop)

In [22]:
# Move offering_id to first column
cols = df.columns.tolist()
cols.remove('offering_id')
df = df[['offering_id'] + cols]

# Group texts by offering_id and combine them
df = df.groupby('offering_id').agg({
    'service': 'mean',
    'cleanliness': 'mean', 
    'overall': 'mean',
    'sleep_quality': 'mean',
    'location': 'mean',
    'rooms': 'mean',
    'value': 'mean',
    'text': lambda x: ' '.join(x)
}).reset_index()


In [30]:
# Import required libraries
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

# Download required NLTK data
import nltk
nltk.download('punkt')
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to clean and preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove numbers and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords and lemmatize
    stop_words = set(stopwords.words('english'))
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    # Join tokens back into text
    return ' '.join(tokens)

# Apply preprocessing to text column
df['text'] = df['text'].apply(preprocess_text)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [31]:
df.head()


,offering_id,service,cleanliness,overall,sleep_quality,location,rooms,value,text
0,72572,4.500000,4.605128,4.328704,4.342222,4.488701,4.238356,4.257653,make fast visit seattle found pioneer regret h...
1,72579,4.034826,4.089109,3.640909,3.804054,4.073446,3.679558,3.915423,great service room clean could use queen bed t...
2,72586,4.016598,4.050209,3.767528,4.090909,4.368932,3.767123,3.892116,beautiful view space needle especially night l...
3,72598,3.303571,3.236364,2.983607,3.325581,3.113208,3.096154,3.142857,hotel need serious update room big bed comfort...
4,73236,4.217391,2.913043,3.173913,3.500000,4.190476,3.190476,3.739130,experience day inn perfect staff great manager...


In [40]:
# Import BM25 implementation from rank_bm25
from rank_bm25 import BM25Okapi

# Tokenize the preprocessed documents for BM25
tokenized_corpus = [doc.split() for doc in df['text']]

# Create BM25 object
bm25 = BM25Okapi(tokenized_corpus)

# Example query
query = "great hotel service"
tokenized_query = query.lower().split()

# Get BM25 scores for the query
doc_scores = bm25.get_scores(tokenized_query)

# Get top 5 most relevant documents
top_n = 5
top_docs = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:top_n]

print("Top 5 most relevant reviews for query '{}':".format(query))
print("\nScores and Evaluations:")
for i, doc_id in enumerate(top_docs):
    print(f"\nRank {i+1}")
    print(f"Score: {doc_scores[doc_id]:.4f}")
    print(f"Hotel ID (offering_id): {df.iloc[doc_id]['offering_id']}")
    
    # Get ratings for recommended hotel
    recommended_ratings = get_avg_ratings(df.iloc[doc_id]['offering_id'])
    
    # Calculate MSE if ratings exist
    if recommended_ratings:
        mse = calculate_ratings_mse(get_avg_ratings(72572), recommended_ratings)  # Using example hotel ID from test case
        print(f"MSE with query hotel ratings: {mse:.4f}")
    
    print(f"Original text:\n{df.iloc[doc_id]['text']}")


Top 5 most relevant reviews for query 'great hotel service':

Scores and Evaluations:

Rank 1
Score: 13.6019
Hotel ID (offering_id): 111507
MSE with query hotel ratings: 0.0801
Original text:
many hotel chicago nice room recent experience ritz bring husband back year returning room midnight called different pillow request took min frustrating however one complain rather keep mind future trip said nothing following day bottled water chocolate covered strawberry sent room along sincere apology manager found pillow debacle idk know returning wonderful hotel every time chicago friendly staff go beyond worldclass hotel beating heart great city quaint rather fully luxurious hq explore everything city offer much within easy moderate walking distance recently visited ritz carlton chicago four season hotel addition amazingly comfortable bed four season known believe room best view city spectacular staff kind helpful special thanks concierge also deca bar staff took great care decadent request h

In [38]:
# Function to calculate MSE between ratings
def calculate_ratings_mse(query_ratings, recommended_ratings):
    aspects = ['service', 'cleanliness', 'overall', 'sleep_quality', 'location', 'rooms', 'value']
    squared_errors = []
    
    for aspect in aspects:
        if aspect in query_ratings and aspect in recommended_ratings:
            error = (query_ratings[aspect] - recommended_ratings[aspect]) ** 2
            squared_errors.append(error)
    
    if not squared_errors:
        return None
    return sum(squared_errors) / len(squared_errors)

# Function to get average ratings for an offering_id
def get_avg_ratings(offering_id):
    hotel_reviews = df[df['offering_id'] == offering_id]
    if len(hotel_reviews) == 0:
        return None
    
    avg_ratings = {}
    for aspect in ['service', 'cleanliness', 'overall', 'sleep_quality', 'location', 'rooms', 'value']:
        avg_ratings[aspect] = hotel_reviews[aspect].mean()
    return avg_ratings

# Example queries with associated offering_ids and their known ratings
test_queries = [
    {
        'query': "great hotel service",
        'offering_id': 72572  # Example offering_id for comparison
    },
    # Add more test queries as needed
]

# Evaluate BM25 recommendations
for test_case in test_queries:
    query = test_case['query']
    query_offering_id = test_case['offering_id']
    
    # Get query hotel's average ratings
    query_ratings = get_avg_ratings(query_offering_id)
    if query_ratings is None:
        print(f"No ratings found for query offering_id: {query_offering_id}")
        continue
        
    # Get BM25 recommendations
    tokenized_query = query.lower().split()
    doc_scores = bm25.get_scores(tokenized_query)
    top_docs = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:5]
    
    # Calculate MSE for each recommended document
    print(f"\nEvaluation for query: '{query}'")
    print(f"Query hotel (offering_id: {query_offering_id}) average ratings:")
    print(query_ratings)
    print("\nRecommended hotels MSE:")
    
    for rank, doc_id in enumerate(top_docs, 1):
        recommended_offering_id = df.iloc[doc_id]['offering_id']
        recommended_ratings = get_avg_ratings(recommended_offering_id)
        
        if recommended_ratings:
            mse = calculate_ratings_mse(query_ratings, recommended_ratings)
            print(f"\nRank {rank} (offering_id: {recommended_offering_id})")
            print(f"MSE: {mse:.4f}")
            print("Average ratings:", recommended_ratings)




Evaluation for query: 'great hotel service'
Query hotel (offering_id: 72572) average ratings:
{'service': np.float64(4.5), 'cleanliness': np.float64(4.605128205128205), 'overall': np.float64(4.328703703703703), 'sleep_quality': np.float64(4.342222222222222), 'location': np.float64(4.488700564971752), 'rooms': np.float64(4.238356164383561), 'value': np.float64(4.25765306122449)}

Recommended hotels MSE:

Rank 1 (offering_id: 111507)
MSE: 0.0801
Average ratings: {'service': np.float64(4.682330827067669), 'cleanliness': np.float64(4.808795411089866), 'overall': np.float64(4.637961335676626), 'sleep_quality': np.float64(4.688607594936709), 'location': np.float64(4.904656319290465), 'rooms': np.float64(4.542116630669547), 'value': np.float64(4.1878557874762805)}

Rank 2 (offering_id: 258705)
MSE: 0.1441
Average ratings: {'service': np.float64(4.824573378839591), 'cleanliness': np.float64(4.89448275862069), 'overall': np.float64(4.7793733681462145), 'sleep_quality': np.float64(4.77344398340